In [1]:
import os

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
data = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')
data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [3]:
data.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [4]:
target = 'HeartDisease'

# Data Preprocessing

In [5]:
for col in data.columns:
  print(f'unique({col}) = {len(data[col].unique())}')

unique(Age) = 50
unique(Sex) = 2
unique(ChestPainType) = 4
unique(RestingBP) = 67
unique(Cholesterol) = 222
unique(FastingBS) = 2
unique(RestingECG) = 3
unique(MaxHR) = 119
unique(ExerciseAngina) = 2
unique(Oldpeak) = 53
unique(ST_Slope) = 3
unique(HeartDisease) = 2


In [6]:
def visualiseHistograms(data: pd.DataFrame, visualizableFields = []):
  n = len(visualizableFields)
  n = n // 2 + (n % 2 != 0)

  plt.figure(figsize = (10, 12))

  for idx, value in enumerate(visualizableFields):
    plt.subplot(n, 2, idx+1)
    data[value].hist()
    plt.title(f"{value} Distribution")
    plt.tight_layout(pad=2)

**Credit**:
https://www.kaggle.com/code/manas28/heart-failure-prediction-pytorch-ann